<a href="https://colab.research.google.com/github/gizelleguerra/genAI_demos/blob/main/Gradio_Resume_GPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resume GPT Tool with Langchain and Gradio Interface

followed tutorial here: https://github.com/RajKKapadia/YouTube-Document-ChatGPT/blob/main/index.py

## Installs

In [1]:
!python3 -m pip install --upgrade langchain faiss-cpu chromadb openai tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 72.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.6/123.6 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.6/62.6 kB 6.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 922.7/922.7 kB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install pypdf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.3/249.3 kB 5.2 MB/s eta 0:00:00


In [3]:
!pip install gradio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 54.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.1/288.1 kB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.7/45.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 2.6 MB/s eta 0:00:00
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4694 sha256=2bfeac86a0c28c557917a84bd1e08d20526059234c1d5a696ca8d211f8433878
  Stored in directory: /root/.cache/pip/wheels/0c/c2/0e/3b9c6845c6a4e

## Imports and Google Drive Mount

In [4]:
# set up google drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
import argparse, json, time, datetime, openai
from pathlib import Path

In [6]:
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma

import gradio as gr

In [7]:
def set_open_ai_key(env_path=None):
  #import json, os
  #from pathlib import Path
  try:
    with open(env_path, "r") as f:
        env_vars = json.load(f)
    os.environ["OPENAI_API_KEY"] = env_vars["OPENAI_API_KEY"]
    openai.api_key = os.environ["OPENAI_API_KEY"]
    #os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')
    openai.Model.list() #test a random command on the openai API
    return True
  except Exception as e:
    print(e)
  return False

## Setup

In [8]:
# setup API key
openai_env_path, openai.api_key = None, None
cwd = Path.cwd()
# resume_path = cwd / "Resumes"
# resume_path.mkdir(exist_ok=True)

openai_env_path = cwd/ "drive/MyDrive/Colab Notebooks/openai.env"
set_open_ai_key(openai_env_path)

True

## Load and Parse Resume Books

In [ ]:
#cwd = Path.cwd()
#output_path = cwd / "drive/MyDrive/Colab Notebooks/Output"

In [9]:
#Set up document loader for pdf resume books
resume_path1 = "/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf"
resume_path2 = "/content/drive/MyDrive/Colab Notebooks/resume_books/GDI2022ResumeBook.pdf"

In [10]:
def load_resumes(path, skip_pages):
  loader = PyPDFLoader(path)
  pages = loader.load_and_split()
  pages_clean = pages[skip_pages:]
  return pages_clean

In [11]:
# actual resumes start on page 2 of this pdf compilation
r1 = load_resumes(resume_path1, 2)

In [12]:
# actual resumes start on page 1 of this pdf compilation
r2 = load_resumes(resume_path2, 1)

In [13]:
r1[0]

Document(page_content='YIN FU(206) 889-7382 ■fuyin1999@gmail.com■linkedin.com/in/yinfu1■https://github.com/fuyin19EDUCATIONNEW YORK UNIVERSITYNew York, NYThe Courant Institute of Mathematical SciencesM.S. in Mathematics in Finance(Sep. 2021 - Dec. 2022)●Coursework:Stochasticcalculus,derivativepricing,quantitativeportfoliotheory,riskmanagement,financialdata science and machine learning, time series analysis, interest rate modelingUNIVERSITY OF WASHINGTONSeattle, WABS in Mathematics(Sep. 2017 – Jun. 2021)●Coursework:Probability, linear algebra, numericalanalysis, statistics, ODEs and PDEs, measure theory●Honors:Magna Cum Laude (Top 3.5%), Dean’s ListEXPERIENCECHINA CONSTRUCTION BANK, NEW YORK BRANCHNew York, NYQuantitative Risk Analyst(Jun. 2022 – Aug. 2022)●Builtacountryriskpredictorleveraginglinearmodels,boosting,randomforestbasedonS&Pdataofeconomics and political factors, and achieved 87.2% in-sample and 78.9% out-of-sample accuracy●Drafted country risk report for the US collaborative

In [14]:
# combine resume books
resumes = r1+r2

In [24]:
len(resumes)

118

### TO DO
add resume names to metadata fields

In [ ]:
#import re

In [16]:
# for resume in resumes:
# #extract name from first line of resume (only up to first delimiter.Muliple types if delimiters tho)- use regex?
# #add as metadata field
#   text = resume.page_content
#   name = re.match("(.*?)/n", text).group().strip('/n') #^([^,])+
#   print(name)
#   resume.metadata["name"] = name

In [17]:
#fake number names until regex is figured out
for i, resume in enumerate(resumes):
  resume.metadata.update({"name":f"name_{i}"})

## Create index

In [35]:
def create_index(resumes) -> None:

    # text_splitter = RecursiveCharacterTextSplitter(
    #     chunk_size = 500,
    #     chunk_overlap  = 10,
    #     length_function = len,
    # )

    text_splitter = CharacterTextSplitter(
        separator='\n',
        chunk_size=500,
        chunk_overlap=10
    )

    texts = text_splitter.split_documents(resumes)

    # embeddings = OpenAIEmbeddings(
    #     openai_api_key=os.getenv('OPENAI_API_KEY')
    # )
    
    embeddings = OpenAIEmbeddings()
    
    #db = Chroma.from_documents(texts, embeddings)

    persist_directory = 'db'

    vectordb = Chroma.from_documents(
        documents=texts,
        embedding=embeddings,
        persist_directory=persist_directory
    )

    vectordb.persist()

In [36]:
create_index(resumes)

FloatProgress(value=0.0, layout=Layout(width='100%'), style=ProgressStyle(bar_color='black'))

### Create Conversation

In [37]:
def create_conversation() -> ConversationalRetrievalChain:

    persist_directory = 'db'

    embeddings = OpenAIEmbeddings(
        openai_api_key=os.getenv('OPENAI_API_KEY')
    )

    db = Chroma(
        persist_directory=persist_directory,
        embedding_function=embeddings
    )

    memory = ConversationBufferMemory(
        memory_key='chat_history',
        return_messages=False
    )

    #add initial prompt to system - use prompt template

    qa = ConversationalRetrievalChain.from_llm(
        llm=ChatOpenAI(),
        chain_type='stuff',
        retriever=db.as_retriever(),
        memory=memory,
        get_chat_history=lambda h: h,
        verbose=True
    )

    return qa

In [38]:
qa = create_conversation()

In [39]:
def add_text(history, text):
    history = history + [(text, None)]
    return history, ""


def bot(history):
    res = qa(
        {
            'question': history[-1][0],
            'chat_history': history[:-1]
        }
    )
    history[-1][1] = res['answer']
    return history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot([], elem_id="chatbot",
                         label='Resume GPT').style(height=750)
    with gr.Row():
        with gr.Column(scale=0.80):
            txt = gr.Textbox(
                show_label=False,
                placeholder="Enter text and press enter",
            ).style(container=False)
        with gr.Column(scale=0.10):
            submit_btn = gr.Button(
                'Submit',
                variant='primary'
            )
        with gr.Column(scale=0.10):
            clear_btn = gr.Button(
                'Clear',
                variant='stop'
            )

    txt.submit(add_text, [chatbot, txt], [chatbot, txt]).then(
        bot, chatbot, chatbot
    )

    submit_btn.click(add_text, [chatbot, txt], [chatbot, txt]).then(
        bot, chatbot, chatbot
    )

    clear_btn.click(lambda: None, None, chatbot, queue=False)

In [40]:
demo.queue(concurrency_count=3)
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e35e1fac79d2cdb232.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


OLD

In [ ]:
# split the documents into chunks
#text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
#texts = text_splitter.split_documents(resumes)

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set small chunk size, just to test.
    chunk_size = 500,
    chunk_overlap  = 10,
    length_function = len,
)

In [ ]:
texts = text_splitter.split_documents(resumes)
print(texts[0])

page_content='YIN FU(206) 889-7382 ■fuyin1999@gmail.com■linkedin.com/in/yinfu1■https://github.com/fuyin19EDUCATIONNEW YORK UNIVERSITYNew York, NYThe Courant Institute of Mathematical SciencesM.S. in Mathematics in Finance(Sep. 2021 - Dec. 2022)●Coursework:Stochasticcalculus,derivativepricing,quantitativeportfoliotheory,riskmanagement,financialdata science and machine learning, time series analysis, interest rate modelingUNIVERSITY OF WASHINGTONSeattle, WABS in Mathematics(Sep. 2017 – Jun.' metadata={'source': '/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf', 'page': 2, 'name': 'name0'}


## Use Vector Stores to create embeddings and preform similarity search

to do: figure out which vector store is best to use - FAISS vs Chroma

In [ ]:
from langchain.vectorstores import FAISS
from langchain.vectorstores import Chroma

### Using FAISS

In [ ]:
faiss_index = FAISS.from_documents(texts, OpenAIEmbeddings())

RateLimitError: ignored

In [ ]:
docs = faiss_index.similarity_search("knows statistics", k=8)

In [ ]:
for doc in docs:
    #print(str(doc.metadata["page"]) + ":", doc.page_content)
    print(str(doc.metadata["page"]) + ":", doc.page_content[:100] + ":", str(doc.metadata["source"]))

In [ ]:
docs[0]

Document(page_content='●\nCoursework:\nprobability, stochastic processes, time\nseries analysis, numerical analysis, statistical\ninference\n●\nAwards\n: School Achievement Prize (top 1% in class)\nWORK EXPERIENCE\nPUBLIC INVESTMENT FUND\nNew York, NY\nMacro Quantitative Research Consultant\n(May 2022 –\nOct 2022)\n●\nBuilt Dynamic Factor Model that forecasted US recession probability with factors constructed based\non economic activities and financial-market indicators (Pseudo R-squared 0.85 in 30-year backtest)\n●', metadata={'source': '/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf', 'page': 5})

## Call OpenAI to use GPT to answer questions based on Resumes

In [ ]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [ ]:
question = "which resumes have stats skills?"

In [ ]:
# make output in json with the following source, page, knows statistics 
#context = " Generate a list of resulting resumes with page numbers and source file information, provide them in JSON format with the following keys: page_id, source_file"
#context = " Generate a list of resulting resumes with their metadata fields, provide them in JSON format with the following keys: page_id, name, source_file"
#context = " Provide resulting resumes in JSON format with the following keys using infomation in the document metadata fields: page_id, source_file"
context = " Provide a list of resulting resumes with their metadata fields in JSON format with the following keys: page_id, name, source_file"

In [ ]:
query = question + context
print(query)

which resumes have stats skills? Generate a list of resulting resumes with their metadata fields, provide them in JSON format with the following keys: page_id, source_file


In [ ]:
chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")

In [ ]:
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': ' The resumes with stats skills are:\n[\n  {\n    "page_id": 0,\n    "source_file": "/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf"\n  },\n  {\n    "page_id": 4,\n    "source_file": "/content/drive/MyDrive/Colab Notebooks/resume_books/GDI2022ResumeBook.pdf"\n  },\n  {\n    "page_id": 8,\n    "source_file": "/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf"\n  },\n  {\n    "page_id": 12,\n    "source_file": "/content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf"\n  }\n]\nSOURCES: /content/drive/MyDrive/Colab Notebooks/resume_books/resume_book_2022.pdf, /content/drive/MyDrive/Colab Note'}

In [ ]:
# Check resumes in results JSON for stats
r1[8]

Document(page_content='The Mathematics in F inance Masters Program \nCourant Institute, New York University \nAcademic Year 2021-2022 \nThe curriculum has four main components: \n1.Financial Theory, Sta tistics and Financial Data Science. These co urses form the co re of the \nprogram, covering topics ranging from equilibrium theory, Black-S choles,  Heath-J arrow- Morton,  \nlinear regressions, co variance matrix estimatio n to modern machine learning techniques and how they \nare used in quantitative finance.\n2.Practical Financial Applications. These classes are taught by indu stry specialists fro m prominent  \nWall Street firms. They emphasize the practical aspects of q uantitative finance,  drawing on the \ninstructo r’s subject matter experience and expertise.\n3.Mathematical Tools. This component provides appropriate mathematical background in areas like \nstochastic calculus an d partial differen tial equations.\n4.Computationa l Skills. These classes prov ide students with a 

In [ ]:
def qa(files, skill, question, n):
    # load documents
    #loader = PyPDFLoader(file)
    #documents = loader.load()

    # split the documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
      # Set small chunk size, just to test.
      chunk_size = 500,
      chunk_overlap  = 10,
      length_function = len,
      )
    
    texts = text_splitter.split_documents(files)

    # select which embeddings we want to use
    embeddings = OpenAIEmbeddings()

    # create the vectorestore to use as the index
    #db = Chroma.from_documents(texts, embeddings)
    faiss_index = FAISS.from_documents(texts, OpenAIEmbeddings())

    # expose this index in a retriever interface
    #retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
    skill_search = {f"knows {skill}"}
    docs = faiss_index.similarity_search(skill_search, k=n)
    
    context = " Provide a list of resulting resumes with their metadata fields in JSON format with the following keys: page_id, name, source_file"
    query = question + context

    # create a chain to answer questions 
    chain = load_qa_with_sources_chain(OpenAI(temperature=0), chain_type="stuff")
    result = chain({"input_documents": docs, "question": query}, return_only_outputs=True)
    print(result['output_text'])

    # qa = RetrievalQA.from_chain_type(
    #     llm=OpenAI(), chain_type=chain_type, retriever=retriever, return_source_documents=True)
    # result = qa({"query": query})
    # print(result['result'])
    return result

In [ ]:
output = qa(resumes, 'stats', )

In [ ]:
# def qa(file, query, chain_type, k):
#     # load document
#     loader = PyPDFLoader(file)
#     documents = loader.load()
#     # split the documents into chunks
#     text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
#     texts = text_splitter.split_documents(documents)
#     # select which embeddings we want to use
#     embeddings = OpenAIEmbeddings()
#     # create the vectorestore to use as the index
#     db = Chroma.from_documents(texts, embeddings)
#     # expose this index in a retriever interface
#     retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": k})
#     # create a chain to answer questions 
#     qa = RetrievalQA.from_chain_type(
#         llm=OpenAI(), chain_type=chain_type, retriever=retriever, return_source_documents=True)
#     result = qa({"query": query})
#     print(result['result'])
#     return result

### Using Chroma

In [ ]:
# create the vectorestore to use as the index
db = Chroma.from_documents(texts, embeddings)
# expose this index in a retriever interface
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 8})

KeyboardInterrupt: ignored

In [ ]:
# create a chain to answer questions 
qa = RetrievalQA.from_chain_type(llm=OpenAI(temperature=0), chain_type="stuff", retriever=retriever, return_source_documents=False)
result = qa({"query": query})
print(result['result'])

In [ ]:
# chain = load_qa_with_sources_chain(llm=OpenAI(), chain_type="stuff")
# chain({"input_documents": docs, "question": query}, return_source_documents=True)

In [ ]:
# db = DeepLake(dataset_path=dataset_path, read_only=True, embedding_function=embeddings)

# retriever = db.as_retriever()
# retriever.search_kwargs['distance_metric'] = 'cos'
# retriever.search_kwargs['k'] = 4

# qa = RetrievalQA.from_chain_type(llm=OpenAI(), chain_type="stuff", retriever=retriever, return_source_documents=False)

# # What was the restaurant the group was talking about called?
# query = input("Enter query:")

# # The Hungry Lobster
# ans = qa({"query": query})

# print(ans)

## Gradio Wrapper

In [ ]:
# Make it pretty with Gradio

In [ ]:
start_sequence = "\nAI:"
restart_sequence = "\nHuman: "

prompt = "The following is a conversation with an AI assistant. The assistant is helpful, creative, clever, and very friendly.\n\nHuman: Hello, who are you?\nAI: I am an AI created by OpenAI. How can I help you today?\nHuman: "


In [ ]:
def openai_create(prompt):
    response = openai.Completion.create(
    model="text-davinci-003",
    prompt=prompt,
    temperature=0.9,
    max_tokens=150,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0.6,
    stop=[" Human:", " AI:"]
    )

    return response.choices[0].text

In [ ]:
def chatgpt_clone(input, history):
    history = history or []
    s = list(sum(history, ()))
    s.append(input)
    inp = ' '.join(s)
    output = openai_create(inp)
    history.append((input, output))
    return history, history


block = gr.Blocks()


with block:
    gr.Markdown("""<h1><center>Build Yo'own ChatGPT with OpenAI API & Gradio</center></h1>
    """)
    chatbot = gr.Chatbot()
    message = gr.Textbox(placeholder=prompt)
    state = gr.State()
    submit = gr.Button("SEND")
    submit.click(chatgpt_clone, inputs=[message, state], outputs=[chatbot, state])

block.launch(debug = True)